# import data

In [1]:
import platform
import pandas as pd
from datetime import datetime, timedelta, time
ohlcv_data = pd.read_pickle('/Users/bokie/code/python/CMS/可转债套利/cms/stock_data_min_new.pkl')
print(ohlcv_data.keys())
for k,v in ohlcv_data.items():
    print(k,' data length: ',len(v))
print('数据起始日期',v.index[0])
print('数据结束日期',v.index[-1])
print('113642.SH数据起始日期',ohlcv_data['113642.SH'].index[0])
print('113642.SH数据结束日期',ohlcv_data['113642.SH'].index[-1])

dict_keys(['123046.SZ', '300587.SZ', '113642.SH', '603185.SH', '110061.SH', '600674.SH', '128081.SZ', '002203.SZ'])
123046.SZ  data length:  31460
300587.SZ  data length:  31460
113642.SH  data length:  13552
603185.SH  data length:  31460
110061.SH  data length:  31460
600674.SH  data length:  31460
128081.SZ  data length:  31460
002203.SZ  data length:  31460
数据起始日期 2021-12-13 09:30:00
数据结束日期 2022-06-28 15:00:00
113642.SH数据起始日期 2022-04-06 09:30:00
113642.SH数据结束日期 2022-06-28 15:00:00


# bond-stock pair

In [2]:
# cbond_pair = ['128081.SZ','002203.SZ']
# part = 1
# convert_price = 9.69
# stock_volume = 100/convert_price #整个回测期间不变


# cbond_pair = ['123046.SZ','300587.SZ']
# part = 1 
# convert_price = 6.73
# stock_volume = 100/convert_price  # 2021-12-13至2022-03-23
# part = 2
# convert_price = 6.74
# stock_volume = 100/convert_price  # 2022-03-24至2022-06-28


# cbond_pair = ['110061.SH','600674.SH']
# part = 1
# convert_price = 9.2
# stock_volume = 100/convert_price #整个回测期间不变


cbond_pair = ['113642.SH','603185.SH']
# part = 1
# convert_price = 145.66
# stock_volume = 100/convert_price  # 2022-04-06至2022-06-02
part = 2
convert_price = 102.61
stock_volume = 100/convert_price  # 2022-06-03至2022-06-28


In [3]:
cbond_pair_price = pd.DataFrame([])
cbond_pair_price = pd.concat([ohlcv_data[cbond_pair[0]]['close'],ohlcv_data[cbond_pair[1]]['close']],axis=1)
cbond_pair_price.columns = cbond_pair
cbond_pair_price = cbond_pair_price.dropna()
if cbond_pair[0] == '128081.SZ' or cbond_pair[0] == '110061.SH':
    cbond_pair_price = cbond_pair_price.loc['2021-12-13 15:00:00':]
elif cbond_pair[0] == '123046.SZ' and part == 1:
    cbond_pair_price = cbond_pair_price.loc['2021-12-13 15:00:00':'2022-03-23 15:00:00']
elif cbond_pair[0] == '123046.SZ' and part == 2:
    cbond_pair_price = cbond_pair_price.loc['2022-03-25 15:00:00':]
elif cbond_pair[0] == '113642.SH' and part == 1:
    cbond_pair_price = cbond_pair_price.loc['2022-04-07 15:00:00':'2022-06-02 15:00:00']
elif cbond_pair[0] == '113642.SH' and part == 2:
    cbond_pair_price = cbond_pair_price.loc['2022-06-03 15:00:00':]


# signal calculate

In [4]:
cbond_return = pd.DataFrame([])

cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
stock_preclose = cbond_pair_price.iloc[0,1] # 初始时刻的stock前收盘价

for index,row in cbond_pair_price.iterrows():
    
    cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
    cbond_return.loc[index,cbond_pair[1]] = row[cbond_pair[1]]/stock_preclose - 1

    if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
        cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
        stock_preclose = row[cbond_pair[1]]
        
# 检查df是否有空值
cbond_return.isnull().sum(axis=0)  # 按列统计
cbond_return.isnull().sum(axis=1) # 按行统计
cbond_return.isnull().sum(axis=1).sum() # 统计整张表

0

In [5]:
pd.set_option('display.max_rows', None)
# pair moving trend: diff = cbond reutrn - stock return 
cbond_return['diff'] = cbond_return[cbond_pair[0]] - cbond_return[cbond_pair[1]]

len(cbond_return)

4058

# grid search

In [6]:
import warnings
warnings.filterwarnings("ignore")
grid_search_res = {}

In [7]:
open_threshold_list = [0.005,0.006,0.007,0.008,0.009,
                        0.01,0.011,0.012,0.013,0.014,0.015,
                        # 0.016,0.017,0.018,0.019,0.020,0.021,0.022 # 0.005 ~ 0.01 ~ 0.015 间隔是0.001
                        ]
# close_threshold_list = [0,0.001,0.002,0.003] #  0 ~ 0.001 ~ 0.002 ~ 0.003
# close_threshold_list = [0,0.000001,0.001,0.002,0.003] #  0 ~ 0.001 ~ 0.002 ~ 0.003
close_threshold_list = [0,0.001,0.002,0.003] #  0 ~ 0.001 ~ 0.002 ~ 0.003
for open_threshold in open_threshold_list[:]:
    for close_threshold in close_threshold_list[:]:
        ######################################################################################################################################################
        # open_threshold = 0.01  # 0.005 ~ 0.01 ~ 0.015 间隔是0.001
        # open_threshold = 0.008
        # close_threshold = 0.001 #  0 ~ 0.001 ~ 0.002 ~ 0.003
        pos_open_flag = 0
        neg_open_flag = 0
        cbond_position = pd.DataFrame([])
        cbond_position.index = cbond_return.index
        for index,row in cbond_return.iterrows():

            if row['diff'] >= open_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff落在A区间，且无仓位，正开
                pos_open_flag = 1
                neg_open_flag = 0
                cbond_position.loc[index,cbond_pair[0]] = -1*pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] >= open_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff落在A区间，且有负仓，平仓再正开
                pos_open_flag=1
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = -1*pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] >= open_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff落在A区间，且有正仓，keep
                pos_open_flag=1
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = -1*pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag

            if row['diff'] > close_threshold and row['diff'] < open_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff 落在B区间，且有正仓，keep
                pos_open_flag=1
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = -1*pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] > close_threshold and row['diff'] < open_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff 落在B区间，且有负仓，平仓
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = neg_open_flag
            elif row['diff'] > close_threshold and row['diff'] < open_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff 落在B区间，且无仓，keep
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = neg_open_flag
            
            if row['diff'] >= -1*close_threshold and row['diff'] <= close_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff 落在C区间，且有正仓，平仓
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] >= -1*close_threshold and row['diff'] <= close_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff 落在C区间，且有负仓，平仓
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = neg_open_flag
            elif row['diff'] >= -1*close_threshold and row['diff'] <= close_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff 落在C区间，且无仓位，keep
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = neg_open_flag
            
            if row['diff'] > -1*open_threshold and row['diff'] < -1*close_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff 落在D区间，且有负仓，keep
                pos_open_flag=0
                neg_open_flag=1
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = -1*neg_open_flag

            elif row['diff'] > -1*open_threshold and row['diff'] < -1*close_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff 落在D区间，且有正仓，平仓
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
            elif row['diff'] > -1*open_threshold and row['diff'] < -1*close_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff 落在D区间，且无仓位，keep
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag

            if row['diff'] <= -1*open_threshold and pos_open_flag==0 and neg_open_flag==0:
                #diff 落在E区间，且无仓位，反开
                pos_open_flag=0
                neg_open_flag=1
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = -1*neg_open_flag
            elif row['diff'] <= -1*open_threshold and pos_open_flag==1 and neg_open_flag==0:
                #diff 落在E区间，且有正仓，平仓反开
                pos_open_flag=0
                neg_open_flag=1
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = -1*neg_open_flag
            elif row['diff'] <= -1*open_threshold and pos_open_flag==0 and neg_open_flag==1:
                #diff 落在E区间，且有负仓，keep
                pos_open_flag=0
                neg_open_flag=1
                cbond_position.loc[index,cbond_pair[0]] = neg_open_flag
                cbond_position.loc[index,cbond_pair[1]] = -1*neg_open_flag

            if index.time()==time(15,0,0): # 每日收盘时必须平仓（时间止损信号）
                pos_open_flag=0
                neg_open_flag=0
                cbond_position.loc[index,cbond_pair[0]] = pos_open_flag
                cbond_position.loc[index,cbond_pair[1]] = pos_open_flag
        ######################################################################################################################################################
        # cbond_position.to_csv('cbond_position.csv')
        df_signal = pd.concat([cbond_pair_price,cbond_return,cbond_position],axis=1)
        df_signal.columns = ['bond_price','stock_price','bond_rtn','stock_rtn','diff','bond_pos','stock_pos']
        # df_signal.to_csv('df_signal.csv')

        # 用来记录交易流水的表格,一边遍历df_signal，一边填入内容到流水表格里

        import numpy as np
        cols = ['balance','signal','bond price','stock price','bond hold','stock hold','commission','tax','reverse open']
        df_trade = pd.DataFrame(
            data = [[0,np.nan,np.nan,np.nan,0,0,0,0,np.nan]],
            columns=cols
            ) 

        balance_last = 0 # 现金账户初始余额

        bond_hold = 0 # 债券持有数量
        stock_hold = 0 # 股票持有数量

        commission_rate = 2/10000 # 佣金买卖万二
        tax_rate = 1/1000 #



        for i in range(len(df_signal)-1):
            if df_signal.values[i][-1] != df_signal.values[i+1][-1]:
                index = df_signal.index[i+1]

            # df_signal的columns
            # ['bond_price','stock_price','bond_rtn','stock_rtn','diff','bond_pos','stock_pos']
                signal = str(int(df_signal.values[i+1][-2]))+str(int(df_signal.values[i+1][-1]))
                
                bond_price = df_signal.values[i+1][0] 

                stock_price = df_signal.values[i+1][1]

                commission = (bond_price + stock_price * stock_volume) * commission_rate

                reverse_open = False #表明这笔交易，是否包含平仓后再反向开仓的操作

                if signal == '00' and stock_hold == stock_volume: #有股票持仓并且需要卖出平仓时，有印花税
                    tax = stock_price * tax_rate * stock_volume
                else:
                    tax = 0

                if signal == '00':
                    balance_last += bond_hold*bond_price
                    balance_last += stock_hold*stock_price
                    
                    bond_hold = 0
                    stock_hold = 0
                    

                if signal == '1-1':
                    '''
                    加一个检测之前持仓的模块,如果之前因为-11的信号有持仓,需要先平掉(即买债卖股票),同时再反向开仓
                    '''
                    reverse_flag = False
                    if bond_hold == -1 and stock_hold == 1*stock_volume:
                        bond_hold = 0
                        stock_hold = 0
                        balance_last -= bond_price 
                        balance_last += stock_price * stock_volume
                        commission *= 2 # 因为平掉之后再反向开仓，手续费double
                        tax = stock_price * tax_rate * stock_volume # 卖股票要收印花税
                        reverse_flag = True
                        reverse_open = True

                    bond_hold += 1
                    stock_hold += (-1 * stock_volume)
                    # stock_hold += stock_volume
                    balance_last -= bond_price
                    balance_last += stock_price * stock_volume
                    
                    if reverse_flag:
                        tax *= 2 #平仓然后反向开仓的时候 已经计算过tax了，直接double
                    else:
                        tax = stock_price * tax_rate * stock_volume # 卖股票要收印花税

                if signal == '-11':
                    '''
                    加一个检测之前持仓的模块,如果之前因为1-1的信号有持仓,需要先平掉(卖债买股票),同时再反向开仓
                    '''
                    if bond_hold == 1 and stock_hold == -1*stock_volume:
                        bond_hold = 0
                        stock_hold = 0
                        balance_last += bond_price
                        balance_last -= stock_price * stock_volume
                        commission *= 2 # 因为平掉之后再反向开仓，手续费double
                                        # 买股票无印花税
                        reverse_open = True
                    bond_hold += -1
                    stock_hold += (1 * stock_volume)
                    # stock_hold += stock_volume
                    balance_last += bond_price
                    balance_last -= stock_price * stock_volume
                

                balance = balance_last

                line = [balance,signal,bond_price,stock_price,bond_hold,stock_hold,commission,tax,reverse_open]
                # df_trade.loc[index,['balance','asset','signal','bond price','stock price','commission','tax']] = line
                # df_trade.loc[index,cols] = [0,0,np.nan,np.nan,np.nan,0,0,0,0]
                df_trade.loc[index,cols] = line

        df_trade['net value'] = df_trade['balance'] + \
                                df_trade['bond price']*df_trade['bond hold'] + \
                                df_trade['stock price']*df_trade['stock hold'] - \
                                df_trade['commission'] - \
                                df_trade['tax']       
        #把交易序列单个时间节点的扩展到整个序列
        # assert False
        df_nv = df_trade[['net value']].iloc[1:]
        df_nv['time'] = df_nv.index
        df_tmp = df_signal[['diff']]
        df_tmp['time'] = df_tmp.index
        #  merge之后再前向往后fillna
        df_nv_full =  pd.merge(df_nv,df_tmp,how='outer').sort_values(by='time').fillna(method='ffill').fillna(0)
        df_nv_full.index = df_tmp.index
        df_nv_full = df_nv_full[['net value']]
        df_trade_2 = df_trade.copy(deep=True)
        rounds = [] #只包含单个来回，每2条交易记录组成一个来回的组
        reverse_rounds = [] # 包含多个来回，信号之间包含平仓同时反向开仓的组
        # for i in range(len(df_trade)-1,-2,-1): 
        
        for i in range(len(df_trade)-1,0,-1): 
        # for i in range(len(df_trade)-1,-1,-1):
            if df_trade_2.iloc[i-1,:]['signal'] == '00':
                round = df_trade_2.iloc[i:,:]
                if len(round)>2:
                    reverse_rounds.append(round)
                else:
                    rounds.append(round)
                df_trade_2 = df_trade_2.iloc[:i]
        rounds.append(df_trade_2.iloc[1:3,:]) # 补完上述切片循环 没有取到的df_trade的第一个来回记录

        num_round = len(rounds)
        for i in reverse_rounds:
            num_round += len(i)-1
        # # print('single round:',num_round)
        # # print('reverse rounds: ',len(reverse_rounds))

        return_in_rounds = []
        win = 0
        lose = 0
        win_profit = 0
        lose_profit = 0
        for i in rounds:
            
            tmp_profit = (i['net value'] - i['net value'].shift(1)).values[-1]
            single_round_cost = (i['bond price'].sum() + (i['stock price']*stock_volume).sum())/4
            
            return_in_round = tmp_profit/single_round_cost
            return_in_rounds.append(return_in_round)
            
            
            if tmp_profit>0:
                win += 1
                win_profit += tmp_profit
            else:
                lose += 1
                lose_profit += tmp_profit
        # # print('win: ',win)
        # # print('lose: ',lose)
        # # print('win profit: ',win_profit)
        # # print('lose profit: ',lose_profit)
        # # print(len(return_in_rounds))

        ''' 
        先把连续反向开仓的信号拆开
        e.g.  1-1 -11 00 ---> 1-1 00 -11 00
        '''

        reverse_rounds_reconstruct = []
        for reverse_round in reverse_rounds:
            reverse_rounds0_reconstruct = []
            for i in range(0,len(reverse_round)-1): #4-2

                # tmp_cols = reverse_rounds[0].columns
                # tmp_index = reverse_rounds[0].iloc[i,:].reset_index()
                m = reverse_round.iloc[i,:].tolist()
                n = reverse_round.iloc[i+1,:].tolist()
                # reverse_rounds0_reconstruct.append(m)
                tmp_balance = m[0] + n[2]*m[4] + n[3]*m[5] 
                tmp_commission = n[6]/2
                tmp_tax = n[7]/2
                tmp_nv =  tmp_balance - tmp_commission - tmp_tax
                
                insert_line  = [tmp_balance,'00',np.nan,np.nan,0,0,tmp_commission,tmp_tax,'-',tmp_nv]
                reverse_rounds0_reconstruct.append(m)
                reverse_rounds0_reconstruct.append(insert_line)
            # reverse_rounds0_reconstruct.append(n)    
            tmp_cols = reverse_round.columns
            reverse_rounds0_reconstruct[-1] = reverse_round.iloc[-1,:].tolist()
            tmp_round = pd.DataFrame(reverse_rounds0_reconstruct[:],columns=tmp_cols)
            tmp_round
            for index,row in tmp_round.iterrows():
                if row['reverse open'] == True:
                    tmp_round.loc[index,['commission','tax']] = tmp_round.loc[index,['commission','tax']]/2
            tmp_round = tmp_round.fillna(method = 'bfill')
            reverse_rounds_reconstruct.append(tmp_round)
            tmp_round['new nv'] = tmp_round['balance'] + tmp_round['bond price']*tmp_round['bond hold'] + tmp_round['stock price']*tmp_round['stock hold'] - tmp_round['commission'] - tmp_round['tax']

        # reverse_rounds_reconstruct[0]
        reverse_rounds_detail = []
        for reverse_round in reverse_rounds_reconstruct:
            # print(len(reverse_round))
            for i in range(0,len(reverse_round)-1,2):
                df_tmp = reverse_round.iloc[i:i+2,:]
                reverse_rounds_detail.append(df_tmp)
        # print(len(reverse_rounds_detail))

        # return_in_rounds = []
        # win = 0
        # lose = 0
        # win_profit = 0
        # lose_profit = 0
        for i in reverse_rounds_detail:
            
            tmp_profit = (i['new nv'] - i['new nv'].shift(1)).values[-1]
            single_round_cost = (i['bond price'].sum() + (i['stock price']*stock_volume).sum())/4
            
            return_in_round = tmp_profit/single_round_cost
            return_in_rounds.append(return_in_round)
            
            
            if tmp_profit>0:
                win += 1
                win_profit += tmp_profit
            else:
                lose += 1
                lose_profit += tmp_profit
        # print('win: ',win)
        # print('lose: ',lose)
        # print('win profit: ',win_profit)
        # print('lose profit: ',lose_profit)
        # print(len(return_in_rounds))

        win_rate = win/(win+lose)
        win_profit_avg = win_profit/win
        lose_profit_avg = lose_profit/lose
        profit_ratio = win_profit_avg/abs(lose_profit_avg)

        total_round = win + lose 

        trade_days = len(df_nv_full.resample('D').last().dropna()) -1 
        round_per_day = total_round/trade_days
        return_per_round = sum(return_in_rounds)/len(return_in_rounds)
        # print('$'*50)
        # print('pair code:',cbond_pair)
        # print('convert price: ',convert_price)
        # print('first trade time',df_trade.index[1])
        # print('last trade time',df_trade.index[-1])
        # print('$'*50)
        # print('win: ',win)
        # print('lose: ',lose)
        # print('win profit: ',win_profit)
        # print('lose profit: ',lose_profit)
        # print('sum of win&lose profit: ',win_profit+lose_profit)
        # print('final net value:',df_trade['net value'].values[-1])
        # print('win rate: ', win_rate)
        # print('profit ratio: ',profit_ratio)
        # print('trade days: ',trade_days)
        # print('total round: ',total_round)
        # print('avg round in a day: ',round_per_day)
        # print('return_per_round: ',return_per_round)
        # print('@'*80)
        df_res = pd.DataFrame(data = [[np.nan]*14],
            columns=['cbond pair','convert price','win','lose','win profit','lose profit',
            'sum of win&lose profit','final net value','win rate',
            'profit ratio','trade days','total round','avg round in a day','return per round'])

        tmp_idx = str(df_trade.index[1]) + ' -> ' + str(df_trade.index[-1])
        tmp_line = [cbond_pair,convert_price,win,lose,win_profit,lose_profit,
                    win_profit+lose_profit,df_trade['net value'].values[-1],win_rate,
                    profit_ratio,trade_days,total_round,round_per_day,return_per_round]

        df_res.loc[tmp_idx,df_res.columns] = tmp_line
        df_res = df_res.dropna()
        df_res['open threshold'] = open_threshold
        df_res['close threshold'] = close_threshold
        
        tmp_key = str(open_threshold) +'@'+ str(close_threshold)
        grid_search_res[tmp_key] = df_res
        
        print(tmp_key)

0.005@0
0.005@0.001
0.005@0.002
0.005@0.003
0.006@0
0.006@0.001
0.006@0.002
0.006@0.003
0.007@0
0.007@0.001
0.007@0.002
0.007@0.003
0.008@0
0.008@0.001
0.008@0.002
0.008@0.003
0.009@0
0.009@0.001
0.009@0.002
0.009@0.003
0.01@0
0.01@0.001
0.01@0.002
0.01@0.003
0.011@0
0.011@0.001
0.011@0.002
0.011@0.003
0.012@0
0.012@0.001
0.012@0.002
0.012@0.003
0.013@0
0.013@0.001
0.013@0.002
0.013@0.003
0.014@0
0.014@0.001
0.014@0.002
0.014@0.003
0.015@0
0.015@0.001
0.015@0.002
0.015@0.003


In [8]:
df_grid_search_res =  pd.concat(list(grid_search_res.values()),axis=0)

In [9]:
df_grid_search_res.head()

,cbond pair,convert price,win,lose,win profit,lose profit,sum of win&lose profit,final net value,win rate,profit ratio,trade days,total round,avg round in a day,return per round,open threshold,close threshold
2022-06-06 09:34:00 -> 2022-06-28 15:00:00,"[113642.SH, 603185.SH]",102.61,53.0,11.0,52.514463,-9.154016,43.360447,44.510431,0.828125,1.190650,16.0,64.0,4.0000,0.004955,0.005,0.000
2022-06-06 09:34:00 -> 2022-06-28 15:00:00,"[113642.SH, 603185.SH]",102.61,64.0,11.0,55.957390,-9.242887,46.714504,48.231216,0.853333,1.040549,16.0,75.0,4.6875,0.004552,0.005,0.001
2022-06-06 09:34:00 -> 2022-06-28 15:00:00,"[113642.SH, 603185.SH]",102.61,76.0,11.0,55.398768,-8.877957,46.520811,46.934043,0.873563,0.903163,16.0,87.0,5.4375,0.003900,0.005,0.002
2022-06-06 09:34:00 -> 2022-06-28 15:00:00,"[113642.SH, 603185.SH]",102.61,99.0,12.0,61.425504,-8.983692,52.441812,53.548323,0.891892,0.828781,16.0,111.0,6.9375,0.003452,0.005,0.003
2022-06-06 09:34:00 -> 2022-06-28 15:00:00,"[113642.SH, 603185.SH]",102.61,40.0,9.0,47.743529,-8.638627,39.104902,39.777357,0.816327,1.243519,16.0,49.0,3.0625,0.005862,0.006,0.000


# return per round surface

In [10]:
df_return_per_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_return_per_round.loc[open_threshold,close_threshold] = row['return per round']
print('max return per round: \n',df_return_per_round.max())
print('min return per round: \n',df_return_per_round.min())
df_return_per_round

max return per round: 
 0.000    0.011500
0.001    0.010964
0.002    0.010352
0.003    0.009873
dtype: float64
min return per round: 
 0.000    0.004955
0.001    0.004552
0.002    0.003900
0.003    0.003452
dtype: float64


,0.000,0.001,0.002,0.003
0.005,0.004955,0.004552,0.003900,0.003452
0.006,0.005862,0.005355,0.004790,0.004221
0.007,0.006503,0.006004,0.005404,0.004892
0.008,0.006547,0.006050,0.005614,0.005270
0.009,0.007119,0.006631,0.006366,0.005890
0.010,0.007956,0.007482,0.007189,0.006673
0.011,0.008701,0.008084,0.007670,0.007442
0.012,0.009232,0.008544,0.008146,0.007842
0.013,0.010386,0.009719,0.009160,0.008857
0.014,0.010575,0.010083,0.009470,0.008991


In [11]:
import plotly.graph_objects as go

x_data = df_return_per_round.columns.to_list() # close threshold

y_data = df_return_per_round.index.to_list() # open threshold

z_data = df_return_per_round.copy(deep=True) # return per round

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="return per round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                 )

fig.show()

# win rate surface

In [12]:
df_win_rate = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_win_rate.loc[open_threshold,close_threshold] = row['win rate']
print('max win rate: \n',df_win_rate.max())
print('min win rate: \n',df_win_rate.min())    
df_win_rate

max win rate: 
 0.000    0.833333
0.001    0.853333
0.002    0.873563
0.003    0.891892
dtype: float64
min win rate: 
 0.000    0.785714
0.001    0.804348
0.002    0.809524
0.003    0.809524
dtype: float64


,0.000,0.001,0.002,0.003
0.005,0.828125,0.853333,0.873563,0.891892
0.006,0.816327,0.842105,0.846154,0.876543
0.007,0.800000,0.823529,0.842105,0.869565
0.008,0.785714,0.804348,0.823529,0.842105
0.009,0.818182,0.837838,0.850000,0.863636
0.010,0.833333,0.848485,0.857143,0.864865
0.011,0.807692,0.821429,0.827586,0.833333
0.012,0.833333,0.840000,0.846154,0.851852
0.013,0.818182,0.826087,0.826087,0.833333
0.014,0.800000,0.809524,0.809524,0.809524


In [13]:
import plotly.graph_objects as go

x_data = df_win_rate.columns.to_list() # close threshold

y_data = df_win_rate.index.to_list() # open threshold

z_data = df_win_rate.copy(deep=True) # win rate

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="win rate", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                 )

fig.show()

# total round surface

In [14]:
df_total_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_total_round.loc[open_threshold,close_threshold] = row['total round']
print('max total round: \n',df_total_round.max())
print('min total round: \n',df_total_round.min())    
df_total_round

max total round: 
 0.000     64.0
0.001     75.0
0.002     87.0
0.003    111.0
dtype: float64
min total round: 
 0.000    20.0
0.001    21.0
0.002    21.0
0.003    21.0
dtype: float64


,0.000,0.001,0.002,0.003
0.005,64.0,75.0,87.0,111.0
0.006,49.0,57.0,65.0,81.0
0.007,45.0,51.0,57.0,69.0
0.008,42.0,46.0,51.0,57.0
0.009,33.0,37.0,40.0,44.0
0.010,30.0,33.0,35.0,37.0
0.011,26.0,28.0,29.0,30.0
0.012,24.0,25.0,26.0,27.0
0.013,22.0,23.0,23.0,24.0
0.014,20.0,21.0,21.0,21.0


In [15]:
import plotly.graph_objects as go

x_data = df_total_round.columns.to_list() # close threshold

y_data = df_total_round.index.to_list() # open threshold

z_data = df_total_round.copy(deep=True) # win rate

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="total round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                 )

fig.show()

# win round surface

In [16]:
df_win_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_win_round.loc[open_threshold,close_threshold] = row['win']
print('max win round: \n',df_win_round.max())
print('min win round: \n',df_win_round.min())    
df_win_round

max win round: 
 0.000    53.0
0.001    64.0
0.002    76.0
0.003    99.0
dtype: float64
min win round: 
 0.000    16.0
0.001    17.0
0.002    17.0
0.003    17.0
dtype: float64


,0.000,0.001,0.002,0.003
0.005,53.0,64.0,76.0,99.0
0.006,40.0,48.0,55.0,71.0
0.007,36.0,42.0,48.0,60.0
0.008,33.0,37.0,42.0,48.0
0.009,27.0,31.0,34.0,38.0
0.010,25.0,28.0,30.0,32.0
0.011,21.0,23.0,24.0,25.0
0.012,20.0,21.0,22.0,23.0
0.013,18.0,19.0,19.0,20.0
0.014,16.0,17.0,17.0,17.0


In [17]:
import plotly.graph_objects as go

x_data = df_win_round.columns.to_list() # close threshold

y_data = df_win_round.index.to_list() # open threshold

z_data = df_win_round.copy(deep=True) # win rate

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="win round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                 )

fig.show()

# lose round surface

In [18]:
df_lose_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_lose_round.loc[open_threshold,close_threshold] = row['lose']
print('max win round: \n',df_lose_round.max())
print('min win round: \n',df_lose_round.min())    
df_lose_round

max win round: 
 0.000    11.0
0.001    11.0
0.002    11.0
0.003    12.0
dtype: float64
min win round: 
 0.000    4.0
0.001    4.0
0.002    4.0
0.003    4.0
dtype: float64


,0.000,0.001,0.002,0.003
0.005,11.0,11.0,11.0,12.0
0.006,9.0,9.0,10.0,10.0
0.007,9.0,9.0,9.0,9.0
0.008,9.0,9.0,9.0,9.0
0.009,6.0,6.0,6.0,6.0
0.010,5.0,5.0,5.0,5.0
0.011,5.0,5.0,5.0,5.0
0.012,4.0,4.0,4.0,4.0
0.013,4.0,4.0,4.0,4.0
0.014,4.0,4.0,4.0,4.0


In [19]:
import plotly.graph_objects as go

x_data = df_lose_round.columns.to_list() # close threshold

y_data = df_lose_round.index.to_list() # open threshold

z_data = df_lose_round.copy(deep=True) # win rate

fig = go.Figure(data=[go.Surface(z=z_data.values,x=x_data,y=y_data)])

fig.update_layout(title="lose round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90),  # 4个位置的距离
                  scene=dict(xaxis_title='X-close-threshold',yaxis_title='Y-open-threshold')
                )

fig.show()

# signal check

In [20]:
print('open threshold: ',open_threshold,'close threshold: ',close_threshold)
df_signal

open threshold:  0.015 close threshold:  0.003


,bond_price,stock_price,bond_rtn,stock_rtn,diff,bond_pos,stock_pos
2022-06-06 09:30:00,136.93,113.72,0.000000,0.000000,0.000000e+00,0.0,0.0
2022-06-06 09:31:00,136.96,113.23,0.000219,-0.004309,4.527919e-03,0.0,0.0
2022-06-06 09:32:00,136.52,113.18,-0.002994,-0.004749,1.754274e-03,0.0,0.0
2022-06-06 09:33:00,136.36,112.81,-0.004163,-0.008002,3.839400e-03,0.0,0.0
2022-06-06 09:34:00,136.70,112.58,-0.001680,-0.010025,8.344932e-03,0.0,0.0
2022-06-06 09:35:00,137.24,113.14,0.002264,-0.005100,7.364177e-03,0.0,0.0
2022-06-06 09:36:00,137.10,113.57,0.001242,-0.001319,2.560539e-03,0.0,0.0
2022-06-06 09:37:00,137.47,113.45,0.003944,-0.002374,6.317873e-03,0.0,0.0
2022-06-06 09:38:00,137.50,113.76,0.004163,0.000352,3.810970e-03,0.0,0.0
2022-06-06 09:39:00,138.09,114.51,0.008471,0.006947,1.524595e-03,0.0,0.0


In [21]:
len(rounds)

21

In [22]:
len(df_trade)-1

42

In [23]:
df_trade.to_csv('tmp_trade.csv')
df_trade

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
0,0.000000,NaN,NaN,NaN,0.0,0.000000,0.000000,0.000000,NaN,NaN
2022-06-06 09:41:00,-24.440604,1-1,138.64,117.18,1.0,-0.974564,0.050568,0.114199,False,-0.164767
2022-06-06 09:45:00,2.022180,00,138.83,115.30,0.0,0.000000,0.050239,0.000000,False,1.971941
2022-06-06 09:52:00,-22.741985,1-1,139.10,117.32,1.0,-0.974564,0.050687,0.114336,False,1.857157
2022-06-06 15:00:00,1.229201,00,143.94,123.10,0.0,0.000000,0.052782,0.000000,False,1.176419
2022-06-07 09:32:00,-19.728911,1-1,145.79,128.09,1.0,-0.974564,0.054124,0.124832,False,1.050245
2022-06-07 15:00:00,3.199811,00,142.42,122.61,0.0,0.000000,0.052382,0.000000,False,3.147429
2022-06-09 09:58:00,31.499689,-11,152.05,126.98,-1.0,0.974564,0.055160,0.000000,False,3.144651
2022-06-09 10:01:00,6.845327,00,147.81,126.37,0.0,0.000000,0.054193,0.123156,False,6.667978
2022-06-09 10:34:00,-15.289864,1-1,143.79,124.83,1.0,-0.974564,0.053089,0.121655,False,6.670583


In [24]:
rounds[-1]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-06 09:41:00,-24.440604,1-1,138.64,117.18,1.0,-0.974564,0.050568,0.114199,False,-0.164767
2022-06-06 09:45:00,2.022180,00,138.83,115.30,0.0,0.000000,0.050239,0.000000,False,1.971941


In [25]:
rounds[-2]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-06 09:52:00,-22.741985,1-1,139.10,117.32,1.0,-0.974564,0.050687,0.114336,False,1.857157
2022-06-06 15:00:00,1.229201,00,143.94,123.10,0.0,0.000000,0.052782,0.000000,False,1.176419


In [26]:
rounds[-3]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-07 09:32:00,-19.728911,1-1,145.79,128.09,1.0,-0.974564,0.054124,0.124832,False,1.050245
2022-06-07 15:00:00,3.199811,00,142.42,122.61,0.0,0.000000,0.052382,0.000000,False,3.147429


In [27]:
rounds[-4]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-09 09:58:00,31.499689,-11,152.05,126.98,-1.0,0.974564,0.055160,0.000000,False,3.144651
2022-06-09 10:01:00,6.845327,00,147.81,126.37,0.0,0.000000,0.054193,0.123156,False,6.667978


In [28]:
rounds[-5]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-09 10:34:00,-15.289864,1-1,143.79,124.83,1.0,-0.974564,0.053089,0.121655,False,6.670583
2022-06-09 15:00:00,7.032798,00,145.03,125.91,0.0,0.000000,0.053547,0.000000,False,6.979251


In [29]:
rounds[-6]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-10 09:36:00,31.495670,-11,143.73,122.38,-1.0,0.974564,0.052599,0.000000,False,6.980199
2022-06-10 10:27:00,9.124106,00,145.42,126.26,0.0,0.000000,0.053694,0.123048,False,8.947364


In [30]:
rounds[-7]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-10 13:30:00,-11.824559,1-1,149.25,131.65,1.0,-0.974564,0.055510,0.128301,False,8.940294
2022-06-10 15:00:00,9.238270,00,150.29,132.60,0.0,0.000000,0.055903,0.000000,False,9.182367


In [31]:
rounds[-8]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-13 09:30:00,33.518875,-11,151.89,130.94,-1.0,0.974564,0.055900,0.000000,False,9.182370
2022-06-13 11:17:00,11.785348,00,152.52,134.20,0.0,0.000000,0.056661,0.130786,False,11.597901


In [32]:
rounds[-9]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-13 14:32:00,34.937909,-11,151.22,131.41,-1.0,0.974564,0.055857,0.000000,False,11.729491
2022-06-13 15:00:00,13.392965,00,150.47,132.29,0.0,0.000000,0.055879,0.128925,False,13.208161


In [33]:
rounds[-10]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-14 09:31:00,36.694892,-11,148.28,128.24,-1.0,0.974564,0.054652,0.000000,False,13.338313
2022-06-14 15:00:00,13.321657,00,144.57,124.36,0.0,0.000000,0.053153,0.121197,False,13.147307


In [34]:
rounds[-11]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-15 09:34:00,38.505372,-11,144.1,122.02,-1.0,0.974564,0.052603,0.000000,False,13.269054
2022-06-15 11:14:00,15.519113,00,142.0,122.12,0.0,0.000000,0.052203,0.119014,False,15.347897


In [35]:
rounds[-12]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-16 09:48:00,-6.021368,1-1,146.08,127.79,1.0,-0.974564,0.054124,0.12454,False,15.340450
2022-06-16 15:00:00,16.531817,00,146.43,127.11,0.0,0.000000,0.054061,0.00000,False,16.477755


In [36]:
rounds[-13]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-17 10:26:00,-3.958091,1-1,146.54,129.34,1.0,-0.974564,0.054518,0.12605,False,16.351248
2022-06-17 15:00:00,15.419491,00,149.17,133.18,0.0,0.000000,0.055792,0.00000,False,15.363699


In [37]:
rounds[-14]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-20 09:34:00,-2.221107,1-1,151.38,137.23,1.0,-0.974564,0.057024,0.133739,False,15.228728
2022-06-20 13:04:00,17.096918,00,150.66,134.77,0.0,0.000000,0.056400,0.000000,False,17.040518


In [38]:
rounds[-15]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-21 09:42:00,37.646746,-11,146.98,129.73,-1.0,0.974564,0.054682,0.000000,False,17.042236
2022-06-21 10:03:00,18.570276,00,148.06,132.35,0.0,0.000000,0.055409,0.128984,False,18.385884


In [39]:
rounds[-16]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-22 10:45:00,39.727778,-11,151.33,133.57,-1.0,0.974564,0.056300,0.000000,False,18.513975
2022-06-22 11:23:00,21.312245,00,146.98,131.92,0.0,0.000000,0.055109,0.128564,False,21.128572


In [40]:
rounds[-17]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-23 09:30:00,41.899290,-11,148.06,130.80,-1.0,0.974564,0.055107,0.000000,False,21.257139
2022-06-23 09:40:00,23.207129,00,148.66,133.36,0.0,0.000000,0.055726,0.129968,False,23.021436


In [41]:
rounds[-18]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-23 10:13:00,43.561695,-11,146.20,129.13,-1.0,0.974564,0.054409,0.000000,False,23.15272
2022-06-23 13:02:00,25.224127,00,146.98,132.00,0.0,0.000000,0.055124,0.128642,False,25.04036


In [42]:
rounds[-19]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-27 14:14:00,44.612741,-11,150.37,134.4,-1.0,0.974564,0.056270,0.000000,False,25.167857
2022-06-27 15:00:00,24.564302,00,150.64,134.0,0.0,0.000000,0.056246,0.130592,False,24.377464


In [43]:
rounds[-20]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-28 09:41:00,46.540408,-11,149.03,130.37,-1.0,0.974564,0.055217,0.000000,False,24.509085
2022-06-28 11:03:00,26.143066,00,151.33,134.35,0.0,0.000000,0.056453,0.130933,False,25.955681


In [44]:
rounds[-21]

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
2022-06-28 13:15:00,8.015680,1-1,152.51,137.89,1.0,-0.974564,0.057379,0.134383,False,25.951305
2022-06-28 13:36:00,28.761605,00,153.54,136.26,0.0,0.000000,0.057267,0.000000,False,28.704338
